In [1]:
!pip install unrar
!pip install rarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import re
import rarfile
import zipfile
import codecs
from sklearn.model_selection import train_test_split

In [55]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPool1D, Dropout, MaxPooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
! gdown --id 1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg
To: /content/hamshahri.rar
100% 873M/873M [00:07<00:00, 118MB/s]


In [5]:
def extract_rar_file(path, extract_path):
  files = []
  with rarfile.RarFile(path, 'r') as rf:
    files = rf.namelist()
    rf.extractall(extract_path)
    
  for file in files:
    path_to_file = fr"{extract_path}\{file}"  

In [6]:
path = 'hamshahri.rar'
extract_path = 'hamshahri'
extract_rar_file(path, extract_path)

In [7]:
def extract_zip_file(path, extract_path):
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

In [8]:
hamshahri_corpus_path = 'hamshahri/hamshahriold/Corpus/Hamshahri-Corpus.zip'
hamshahri_corpus_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus'
extract_zip_file(hamshahri_corpus_path, hamshahri_corpus_extract_path)

In [9]:
HAMSHAHRI_CORPUS_PATH = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus/Hamshahri-Corpus.txt'

In [10]:
%%time
with codecs.open(HAMSHAHRI_CORPUS_PATH, "r", "utf-8") as text:
    sentences = []
    pattern = re.compile(".Cat")
    s = ' '
    for line in text:
      for match in re.finditer(pattern, line):
        sentences.append((line,s))
        s = ' '
      s += line         

CPU times: user 1min, sys: 2.45 s, total: 1min 2s
Wall time: 1min 4s


In [11]:
category_punctuation = [re.sub('\.Cat\t', '', line[0]) for line in sentences]
sentences_punctuation = [line[1].replace('\n', '') for line in sentences] 
sentences_punctuation = [line.replace('\r', '') for line in sentences_punctuation] 
sentences_punctuation = [re.sub('\.Cat\s\w+', '', sentence) for sentence in sentences_punctuation]
sentences_punctuation = [re.sub('\.DID\\t\d+S\d+.Date\\t\d+\\\\\d+\\\\\d+', '', sentence) for sentence in sentences_punctuation]

In [12]:
print(sentences_punctuation[1])

  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين ن

In [13]:
corpus = []
for i in range(len(sentences)):
  corpus.append({'text': sentences_punctuation[i], 'category': category_punctuation[i-1]})

In [14]:
corpus[1]

{'category': 'adabh\r\n',
 'text': '  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پاي

In [15]:
df = pd.DataFrame(corpus[1:], columns=['text', 'category'])

In [16]:
df.head()

,text,category
0,در زندگي گروهي از طريق هنر نگاهي به نمايشگاه...,adabh\r\n
1,هنري جهان نمايشگاه هنر در خدمت ديكتاتورها نم...,adabh\r\n
2,نگارخانه ها گالري گلستان: نمايشگاه طرح و ساخ...,adabh\r\n
3,را جدي بگيريم مطالعه اي مقدماتي پيرامون نقش ...,ejtem\r\n
4,سياه و غباري كه سترده نمي شود... اشاره; به ر...,elmfa\r\n


In [17]:
len(df)

165261

In [18]:
df.to_csv('data.csv')

# **Read Dataset**

In [19]:
data = pd.read_csv('data.csv')

In [20]:
data = data.iloc[: , 1:]

In [21]:
data.head(3)

,text,category
0,در زندگي گروهي از طريق هنر نگاهي به نمايشگاه...,adabh\r\n
1,هنري جهان نمايشگاه هنر در خدمت ديكتاتورها نم...,adabh\r\n
2,نگارخانه ها گالري گلستان: نمايشگاه طرح و ساخ...,adabh\r\n


### **Read StopWords**

In [22]:
hamshahri_stopwords_path = 'hamshahri/hamshahriold/Corpus/PersianStopWords.zip'
hamshahri_stopwords_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-stopwords'
extract_zip_file(hamshahri_stopwords_path, hamshahri_stopwords_extract_path)
HAMSHAHRI_STOPWORDS_PATH = '/content/hamshahri/hamshahriold/Corpus/hamshahri-stopwords/PersianStopWords.txt'

In [23]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [24]:
stopwords = read_stop_words(HAMSHAHRI_STOPWORDS_PATH)
print(len(stopwords))

796


In [25]:
# Convert pandas to numpy
np_data = data["text"].values

In [26]:
np_data[0]

'  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين 

**Install hazm library**

In [27]:
# install hazm library
!pip install hazm
from hazm import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.1 MB/s 
     |████████████████████████████████| 233 kB 45.1 MB/s 
     |████████████████████████████████| 1.4 MB 49.2 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=e50a3d9ee26a0447f99ed19c77d564c1b50c98ea398d6afa3f8efc64e3ffec31
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154942 sha256=967b2c6f318756419d920175cc6413892d84db74ba535082a04e34fc7408a7da
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [28]:
word_tokenized = [word_tokenize(t) for t in np_data]

In [29]:
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [34]:
del word_tokenized
del np_data

**category tokenizaer**

In [43]:
category_tokenizer = Tokenizer()
category_tokenizer.fit_on_texts(data['category'])
category_data = category_tokenizer.texts_to_sequences(data['category'])

In [44]:
num_categories = len(category_tokenizer.word_index)+1

In [72]:
category_tokenizer.word_index

{"'coevulea'": 173,
 "'harrisoniana'": 141,
 '1963': 314,
 '1991': 268,
 '2001': 210,
 '216': 233,
 '22': 115,
 '267': 236,
 'abksh\r': 33,
 'adabh\r': 12,
 'adarman\r': 71,
 'aeqts\r': 18,
 'akhar\r': 4,
 'amethystoglossa': 174,
 'art\r': 60,
 'artw\r': 25,
 'aurantiaca': 165,
 'axrooz\r': 36,
 'bankb\r': 28,
 'bazar\r': 43,
 'books\r': 47,
 'bowringiana': 112,
 'bruchor': 196,
 'busiw\r': 67,
 'cartoon\r': 56,
 'cat': 91,
 'catagen': 220,
 'catalog': 198,
 'catalyst': 163,
 'cataracts': 252,
 'catastrofica': 326,
 'catch': 98,
 'categorization': 203,
 'category': 169,
 'cateh': 304,
 'catera': 247,
 'caterpillars': 182,
 'catholic': 123,
 'cathy': 205,
 'cato': 97,
 'cats': 313,
 'catskill': 189,
 'cattlega': 145,
 'cattleya': 88,
 'catzung': 235,
 'cima': 208,
 'cinama\r': 69,
 'cinew\r': 59,
 'city\r': 52,
 'cradle': 312,
 'cradleص': 301,
 'cts': 245,
 'cultw\r': 24,
 'cyclizin': 230,
 'donya\r': 74,
 'earth\r': 53,
 'econo\r': 41,
 'econw\r': 22,
 'ejtem\r': 9,
 'elmfa\r': 14,
 'e

In [45]:
Y = [to_categorical(category, num_categories) for category in category_data]

In [70]:
Y[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

**text tokenizer**

In [48]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(word_tokenized_filtered)

In [49]:
vocab_size = len(tokenizer.word_index)+1

In [50]:
data = tokenizer.texts_to_sequences(word_tokenized_filtered)

In [51]:
maxlen = 500
data = pad_sequences(data, maxlen = maxlen)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.3, random_state=42)

### **Neural Network**

In [57]:
embedding_dim = 50

In [62]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(num_categories, activation='softmax'))

In [63]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           27602850  
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 32)           4832      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 32)          0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 351)               35451     
                                                                 
Total params: 27,696,333
Trainable params: 27,696,333
Non-trainable params: 0
__________________________________________

In [65]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

ValueError: ignored

In [68]:
del data